# Imports and DB connection

In [1]:
import os
import psycopg2
from buspkg import DBFunctions as db

In [2]:
conn = db.GetDefaultConnection()
cur = conn.cursor()

## Create Tables

This notebook works more iteratively throughout the project. As more data is collected, different tables must be created or altered.

If a table must be altered to accomodate new data, the SQL command to alter it will be executed but not commited in git. Instead, the create table command will be updated and the update will be logged. This was a decision to enhance code reproducibility but still keep track of all updates.

### Create Company Info table

- Create table

In [3]:
query = """
    CREATE TABLE IF NOT EXISTS 
        CompanyInfo (
            cnpj VARCHAR(20) PRIMARY KEY,
            name VARCHAR(255)
        );
"""
db.ExecuteQuery(query, cur, conn)
conn.commit()

### Create Trips table

- Create table

In [4]:
query = """
    CREATE TABLE IF NOT EXISTS 
        RegularTrips (
            id VARCHAR(10),
            cnpj VARCHAR(20),
            plate VARCHAR(10),
            line VARCHAR(10),
            tripType VARCHAR(20),
            expectedTripDate DATE,
            expectedTripHour TIME,
            tripStartDateTime TIMESTAMP NOT NULL,
            tripEndDateTime TIMESTAMP,
            lineDirection BOOL,
            latitude REAL,
            longitude REAL,
            pdop REAL,
            imei VARCHAR(20),
            transbordo BOOL,
            transbordoCode VARCHAR(20),
            PRIMARY KEY (tripStartDateTime, id)
        ) 
    PARTITION BY RANGE (tripStartDateTime);
"""

db.ExecuteQuery(query, cur, conn)
conn.commit()

In [5]:
firstYear = 2019
lastYear = 2024
firstMonth = 1
lastMonth = 12

In [6]:
for year in range(firstYear, lastYear+1):
    for month in range(firstMonth, lastMonth+1):
        start = f"{year}-{month:02d}-01"
        end = f"{year + month//12}-{((month)%12 + 1):02d}-01"

        query = f"""
            CREATE TABLE IF NOT EXISTS
                RegularTrips_{year}_{month}
            PARTITION OF
                RegularTrips
            FOR VALUES FROM ('{start}') TO ('{end}')
        """

        db.ExecuteQuery(query, cur, conn)
conn.commit()